In [ ]:
# 패키지 임포트
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm_notebook

In [ ]:
res_table = pd.read_csv("./datas/서울시 착한가격음식점(지역추가).csv", encoding='CP949',index_col=0)
res_table

In [ ]:
search_res = []
for i in res_table.index :
    addr = []
    addr.append(res_table['행정구'][i])
    addr.append(res_table['행정동'][i])
    addr.append(res_table['업소명'][i])
    str = " ".join(addr)
    search_res.append(str)

In [ ]:
search_res

In [ ]:
res_table['검색명'] = search_res

In [ ]:
base_url='https://map.naver.com/v5/?c=14127676.3957465,4523397.9832608,15,0,0,0,dh'

In [ ]:
driver = webdriver.Chrome('./driver/chromedriver')
driver.get(base_url)
time.sleep(2)

In [ ]:
url_list = []

In [ ]:
for res in tqdm_notebook(res_table['검색명']) :
    try :
        search_xpath = '/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input'
        element = driver.find_element(By.XPATH, search_xpath)
        time.sleep(0.5)
        element.clear()
        element.send_keys(res)
        element.send_keys("\n")
        time.sleep(3)
        
        url = driver.current_url
        if(url.split('/').__contains__('place')) :
            u = url.split('/')[-1].split('?')[0]
            url_list.append(u)
        else :
            driver.switch_to.frame('searchIframe')
            time.sleep(1)
            
            target_xpath = '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[2]/a[1]/div/div/span[1]'
            config = driver.find_elements(By.XPATH, target_xpath)
            if len(config) == 0 :
                target_xpath = '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[1]/a[1]/div/div/span[1]'
            else :
                target_xpath = '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[2]/a[1]/div/div/span[1]'
                
            element = driver.find_element(By.XPATH, target_xpath)
            element.click()
            time.sleep(2)

            url = driver.current_url
            u = url.split('/')[-1].split('?')[0]
            url_list.append(u)
            time.sleep(1)

            driver.switch_to.default_content()
            time.sleep(1)
                
    except :
        print('error',res)
        url_list.append('-')
        driver.switch_to.default_content()
        time.sleep(1)

In [ ]:
driver.close()

In [ ]:
res_table['업소URL'] = url_list

In [ ]:
res_table.to_csv('./datas/서울시 착한가격음식점(업소URL추가).csv', encoding='CP949' )